# <span style="color: #4daafc">Legal Case Similarity Detection - Summarize translated documents</span>
- [Load data](#load-data)
- [Summarize translated documents](#summarize-translated-documents)
- [Save data](#save-data)

# Prepare python environment

In [1]:
from utils.file_utils import load_file, save_file
from utils.df import df_shape, concat_df_cols, split_df_col
import tiktoken
from langchain.schema import Document
import numpy as np
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from sklearn.preprocessing import MultiLabelBinarizer
import ast
import re
from dotenv import load_dotenv
import os
import boto3
from botocore.config import Config
from langchain_aws.chat_models import ChatBedrock

# Load data

In [2]:
f_path = 'data/translated_data_ar_ap_100.xlsx'
df = load_file(file_name=f_path)

Successfully loaded DataFrame from data/translated_data_ar_ap_100.xlsx


In [3]:
df_shape(df)
display(df.head(10))

Data shape: 100 rows x 7 columns


,case_number,procedure_name,case_date,case_link,document_body,document_body_english_1,document_body_english_2
0,1108/97,"ע""א 1108/97 מרחיב אביב נ. מדינת ישראל",1997-05-11,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בש""פ 97 / 1108 בפני: כבוד הר...",In the Supreme Court of Israel Case No. 97/110...,NaN
1,4477/00,"ע""א 4477/00 לודמילה וורוביוב נ. היועצ המשפטי ל...",2000-07-06,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בה""נ 4477/00 בפני כבוד נשיא ...","In the Supreme Court of Israel, HCJ 4477/00 Be...",NaN
2,1890/16,"ע""פ 1890/16",2017-03-09,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 1890/16 בבית המשפט העליון בשב...","In this case, the Hebrew text is a translation...",NaN
3,7176/04,"ע""פ 7176/04 ירונ תלמי נ. מדינת ישראל",2006-02-02,https://supremedecisions.court.gov.il/Verdicts...,"פסק-דין בתיק ע""פ 7176/04 בבית המשפט העליון בשב...",In the case of Appellant Yaron Telmi v. State ...,NaN
4,3766/12,"ע""א 3766/12",2012-06-17,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""א 3766/12 בבית המשפט העליון בירוש...",The decision in Case A3766/12 at the Supreme C...,NaN
5,8178/12,"ע""א 8178/12 עו""ד צבי סלנט נ. יונתנ גוטליב",2014-11-12,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""א 8178/12 בבית המשפט העליון בשבתו...",In the case of Appeal No. 8178/12 at the Supre...,NaN
6,3015/09,"ע""פ 3015/09 מדינת ישראל נ. פואד קדיח",2010-07-20,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 3015/09 בבית המשפט העליון בשב...",The State of Israel appeals the sentence issue...,NaN
7,4272/05,"ע""פ 4272/05 אמיר חג'וג' נ. מדינת ישראל",2006-01-04,https://supremedecisions.court.gov.il/Verdicts...,"פסק-דין בתיק ע""פ 4272/05 בבית המשפט העליון בשב...",In the case of Appeal No. 4272/05 at the Supre...,NaN
8,10467/08,"ע""א 10467/08 עומר חג'אזי נ. אדיב עיסא דיאב",2010-11-03,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 10467/08 בבית המשפט העליון בש...",The case of Appeal No. 10467/08 at the Supreme...,he registered a cautionary note or completed ...
9,3330/11,"ע""א 3330/11 אגד אגודה שיתופית לתחבורה בישראל ב...",2011-11-17,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 3330/11 בבית המשפט העליון בשב...",In the case of Appeal No. 3330/11 at the Supre...,NaN


Concatenate document body, translated to engilish columns. <br>
**Reminder**: Excel file has a limit of 32K chars per cell. When we store the data in Excel file, we split the translated english text into multiple columns in order to avoid text truncation above 32K chars. <br>
When we load the the data from excel, we join back the the splitted columns into a single DataFrame column.

In [4]:
df_concat = concat_df_cols(df, col_prefix='document_body_english')

In [5]:
df_concat

,case_number,procedure_name,case_date,case_link,document_body,document_body_english
0,1108/97,"ע""א 1108/97 מרחיב אביב נ. מדינת ישראל",1997-05-11,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בש""פ 97 / 1108 בפני: כבוד הר...",In the Supreme Court of Israel Case No. 97/110...
1,4477/00,"ע""א 4477/00 לודמילה וורוביוב נ. היועצ המשפטי ל...",2000-07-06,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בה""נ 4477/00 בפני כבוד נשיא ...","In the Supreme Court of Israel, HCJ 4477/00 Be..."
2,1890/16,"ע""פ 1890/16",2017-03-09,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 1890/16 בבית המשפט העליון בשב...","In this case, the Hebrew text is a translation..."
3,7176/04,"ע""פ 7176/04 ירונ תלמי נ. מדינת ישראל",2006-02-02,https://supremedecisions.court.gov.il/Verdicts...,"פסק-דין בתיק ע""פ 7176/04 בבית המשפט העליון בשב...",In the case of Appellant Yaron Telmi v. State ...
4,3766/12,"ע""א 3766/12",2012-06-17,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""א 3766/12 בבית המשפט העליון בירוש...",The decision in Case A3766/12 at the Supreme C...
...,...,...,...,...,...,...
95,774/03,"ע""פ 774/03 אסתר כהנ נ. מדינת ישראל",2003-02-04,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""פ 774/03 בבית המשפט העליון ע""פ 77...",In the case of Appeal No. 774/03 at the Suprem...
96,6873/19,"ע""א 6873/19 אגודת המימ השיתופית בעמק חרוד בע""מ...",2020-11-30,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 6873/19 בבית המשפט העליון בשב...",The Supreme Court of Israel has issued a judgm...
97,3238/16,"ע""פ 3238/16 אלינה מלר נ. מדינת ישראל",2017-06-19,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""פ 3238/16 בבית המשפט העליון בשבתו...",In the case of Appellant Elina Meller v. State...
98,4446/20,"ע""פ 4446/20 עדי קוכמנ נ. מדינת ישראל",2020-07-14,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 4446/20 בבית המשפט העליון ע""פ...",The appeal is against the decision of the Jeru...


# Summarize translated documents

### Init langchain AWS Bedrock chat model

In [6]:
# load environment variables from the .env file
load_dotenv()

True

In [8]:
# define params - model id, region, keys, etc.
region_name = 'us-west-2'
model_id = 'anthropic.claude-3-5-sonnet-20241022-v2:0'
endpoint_url = 'https://bedrock-runtime.us-west-2.amazonaws.com'

# get secret keys from environment variables
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# timeout configuration
timeout_config = Config(connect_timeout=30, read_timeout=120)

# initialize Bedrock client
bedrock_client = boto3.client('bedrock-runtime', region_name=region_name, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, endpoint_url=endpoint_url, config=timeout_config)

# initialize the LangChain LLM Chat Bedrock
summarization_llm = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs={
        "max_tokens": 64*1024,
        "temperature": 0.3
    }
)

### Define summarization prompt for LLM

In [9]:
summarization_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    You are an expert legal analyst. Analyze the following legal document and provide a structured short summary focusing on key elements:
    
    1. Case Number: legal case number
    2. Case Type: Specify whether it is a civil, criminal, administrative, or other type of case.
    3. Legal Issues: Highlight the primary legal issues or questions addressed in the case.
    4. Summary of Facts: Summarize the main facts and key points of the case in less than 10 bullet points.
    5. Court Decision: Clearly state the outcome of the case.
    6. Relevant Laws and Precedents: List any significant laws, statutes, or past court decisions referenced.

    Important Instructions:
    - Do not include specific dates or places in the summary.
    - If information for a specific bullet point is not present in the document, exlcude that section from the summary.
    - **Do not include any notes, disclaimers, or meta-comments** such as "No information available" or "Section excluded."
    - Focus on legal concepts, case type, legal issues, facts, and the court's decision.
   
    Document:
    {text}   
    """
)

### Define summarization chain

In [10]:
summarization_chain = summarization_prompt | summarization_llm

In [11]:
def summarize_text(i_text):
    """
    Summarizes the input text by invoking the summarization chain.
    """
    res = summarization_chain.invoke({"text": i_text})
    return res.content

### Summarize documents

Example:

In [12]:
res = summarize_text(df_concat.iloc[94]['document_body_english'])
print(res)

1. Case Number: 5668/13

2. Case Type: Criminal Appeal

3. Legal Issues:
- Appeal against sentencing in a Ponzi scheme fraud case
- Application of Amendment 113 to the Penal Law
- Classification of multiple offenses as single or multiple events
- Appropriateness of fines alongside prison sentences

4. Summary of Facts:
- Licensed stockbroker defrauded investors of approximately 57 million NIS
- Operated a sophisticated Ponzi scheme over five years
- Promised high returns on foreign currency investments
- Used investor funds for personal luxury lifestyle
- Created false documents to maintain the scheme
- Affected 84 investors through theft and fraud
- Some victims were elderly, ill, or personal acquaintances

5. Court Decision:
- Original sentence of 12 years imprisonment upheld
- 18 months probation maintained
- Fine of 300,000 NIS or 9 months imprisonment in lieu thereof
- Ordered compensation to complainants up to 258,000 NIS per victim

6. Relevant Laws and Precedents:
- Amendment 1

Apply summarization to the whole DataFrame:

In [13]:
# apply the summarization function to 'document_body_english' column
df_summarized = df_concat.copy()
df_summarized['document_body_eng_sum'] = df_summarized['document_body_english'].apply(summarize_text)

In [14]:
df_summarized

,case_number,procedure_name,case_date,case_link,document_body,document_body_english,document_body_eng_sum
0,1108/97,"ע""א 1108/97 מרחיב אביב נ. מדינת ישראל",1997-05-11,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בש""פ 97 / 1108 בפני: כבוד הר...",In the Supreme Court of Israel Case No. 97/110...,1. Case Number: 97/1108\n\n2. Case Type: Admin...
1,4477/00,"ע""א 4477/00 לודמילה וורוביוב נ. היועצ המשפטי ל...",2000-07-06,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בה""נ 4477/00 בפני כבוד נשיא ...","In the Supreme Court of Israel, HCJ 4477/00 Be...",\nCase Number: HCJ 4477/00\n\nCase Type: Admin...
2,1890/16,"ע""פ 1890/16",2017-03-09,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 1890/16 בבית המשפט העליון בשב...","In this case, the Hebrew text is a translation...",1. Case Number: 1890/16\n\n2. Case Type: Crimi...
3,7176/04,"ע""פ 7176/04 ירונ תלמי נ. מדינת ישראל",2006-02-02,https://supremedecisions.court.gov.il/Verdicts...,"פסק-דין בתיק ע""פ 7176/04 בבית המשפט העליון בשב...",In the case of Appellant Yaron Telmi v. State ...,Case Number: T.P. 2328/01\n\nCase Type: Crimin...
4,3766/12,"ע""א 3766/12",2012-06-17,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""א 3766/12 בבית המשפט העליון בירוש...",The decision in Case A3766/12 at the Supreme C...,1. Case Number: A3766/12\n\n2. Case Type: Civi...
...,...,...,...,...,...,...,...
95,774/03,"ע""פ 774/03 אסתר כהנ נ. מדינת ישראל",2003-02-04,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""פ 774/03 בבית המשפט העליון ע""פ 77...",In the case of Appeal No. 774/03 at the Suprem...,"1. Case Number: Appeal No. 774/03, Appeal No. ..."
96,6873/19,"ע""א 6873/19 אגודת המימ השיתופית בעמק חרוד בע""מ...",2020-11-30,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 6873/19 בבית המשפט העליון בשב...",The Supreme Court of Israel has issued a judgm...,1. Case Number: 6873/19\n\n2. Case Type: Civil...
97,3238/16,"ע""פ 3238/16 אלינה מלר נ. מדינת ישראל",2017-06-19,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""פ 3238/16 בבית המשפט העליון בשבתו...",In the case of Appellant Elina Meller v. State...,1. Case Number: 3238/16 - Z'\n\n2. Case Type: ...
98,4446/20,"ע""פ 4446/20 עדי קוכמנ נ. מדינת ישראל",2020-07-14,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 4446/20 בבית המשפט העליון ע""פ...",The appeal is against the decision of the Jeru...,1. Case Number: M.I. 6904-03-20\n\n2. Case Typ...


Split column *document_body_english*:

In [15]:
df_sum_split = split_df_col(df_summarized, col='document_body_english')
df_sum_split = df_sum_split.drop(columns=['document_body_english'])

In [16]:
df_sum_split

,case_number,procedure_name,case_date,case_link,document_body,document_body_eng_sum,document_body_english_1,document_body_english_2
0,1108/97,"ע""א 1108/97 מרחיב אביב נ. מדינת ישראל",1997-05-11,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בש""פ 97 / 1108 בפני: כבוד הר...",1. Case Number: 97/1108\n\n2. Case Type: Admin...,In the Supreme Court of Israel Case No. 97/110...,NaN
1,4477/00,"ע""א 4477/00 לודמילה וורוביוב נ. היועצ המשפטי ל...",2000-07-06,https://supremedecisions.court.gov.il/Verdicts...,"בבית המשפט העליון בה""נ 4477/00 בפני כבוד נשיא ...",\nCase Number: HCJ 4477/00\n\nCase Type: Admin...,"In the Supreme Court of Israel, HCJ 4477/00 Be...",NaN
2,1890/16,"ע""פ 1890/16",2017-03-09,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 1890/16 בבית המשפט העליון בשב...",1. Case Number: 1890/16\n\n2. Case Type: Crimi...,"In this case, the Hebrew text is a translation...",NaN
3,7176/04,"ע""פ 7176/04 ירונ תלמי נ. מדינת ישראל",2006-02-02,https://supremedecisions.court.gov.il/Verdicts...,"פסק-דין בתיק ע""פ 7176/04 בבית המשפט העליון בשב...",Case Number: T.P. 2328/01\n\nCase Type: Crimin...,In the case of Appellant Yaron Telmi v. State ...,NaN
4,3766/12,"ע""א 3766/12",2012-06-17,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""א 3766/12 בבית המשפט העליון בירוש...",1. Case Number: A3766/12\n\n2. Case Type: Civi...,The decision in Case A3766/12 at the Supreme C...,NaN
...,...,...,...,...,...,...,...,...
95,774/03,"ע""פ 774/03 אסתר כהנ נ. מדינת ישראל",2003-02-04,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""פ 774/03 בבית המשפט העליון ע""פ 77...","1. Case Number: Appeal No. 774/03, Appeal No. ...",In the case of Appeal No. 774/03 at the Suprem...,NaN
96,6873/19,"ע""א 6873/19 אגודת המימ השיתופית בעמק חרוד בע""מ...",2020-11-30,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 6873/19 בבית המשפט העליון בשב...",1. Case Number: 6873/19\n\n2. Case Type: Civil...,The Supreme Court of Israel has issued a judgm...,NaN
97,3238/16,"ע""פ 3238/16 אלינה מלר נ. מדינת ישראל",2017-06-19,https://supremedecisions.court.gov.il/Verdicts...,"החלטה בתיק ע""פ 3238/16 בבית המשפט העליון בשבתו...",1. Case Number: 3238/16 - Z'\n\n2. Case Type: ...,In the case of Appellant Elina Meller v. State...,NaN
98,4446/20,"ע""פ 4446/20 עדי קוכמנ נ. מדינת ישראל",2020-07-14,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 4446/20 בבית המשפט העליון ע""פ...",1. Case Number: M.I. 6904-03-20\n\n2. Case Typ...,The appeal is against the decision of the Jeru...,NaN


# Save data

In [17]:
# save to file
f_path_summarized = f_path.replace("translated", "sum_trans")
save_file(df_sum_split, f_path_summarized)

DataFrame successfully saved to data/sum_trans_data_ar_ap_100.xlsx
